# ArcGIS Online Replace Layer in a Web Map

Description: The user will provide the layer they wish to replace, and the layer they wish to replace it with. Then the notebook will loop through all the maps searching for the provided layer, and it will replace it with the other provided layer.
    
Created on: 5/21/24

Purpose: This is to streamline the ability to replace layers in webmaps in ArcGIS Online. 

Authored By: Joe Guzi

Previous Production Date: 7/3/24

Production Date: 7/8/24

Note: You can use the ArcGIS Online Web Map Services Audit notebook, below, to get a preview of all of the maps that will be updated by this notebook. Simply run the notebook then filter on the layer you want to replace, and you will have an inventory of all of the maps that will be updated: https://www.arcgis.com/home/item.html?id=72ce7ff61fc5480d850ed68de29f1d9c 

--- 7/3/24 - Minor update for proofreading.

--- 7/8/24 - Added sign in option "Connect with current AGO Credentials to this organization" to allow the notebook to be uploaded to AGO as a hosted notebook and run from AGO.

## Code inspired by:

ArcGIS Online WAB Widget Audit Notebook - https://www.arcgis.com/home/item.html?id=f2609c3d733945c58297c9b96cd1998f

ArcGIS Online Web Map Services Audit Notebook - https://www.arcgis.com/home/item.html?id=72ce7ff61fc5480d850ed68de29f1d9c

ArcGIS Online Basemap Replacement Notebook - https://www.arcgis.com/home/item.html?id=90cc2d7419f947d3ae55b44fd3090b24

https://github.com/Esri/arcgis-python-api/blob/conference_talks/talks/DevSummit2024/Managing_Web_Maps_and_Layers_Using_ArcGIS_API_for_Python/Authoring_and_Editing_Webmap.ipynb  

https://community.esri.com/t5/arcgis-api-for-python-questions/reorder-webmap-layers-with-python/td-p/797542 

https://developers.arcgis.com/python/samples/using-and-updating-GIS-content/ 


# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

print("Define Variables")
LayerURLToReplace = ""
LayerIDToReplace = ""
ReplacementLayerURL = ""
ReplacementLayerItemIO = ""
s = ""
print("Define Variables Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise
Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password, verify_cert=False)
print(gis)
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Layer you want to replace
Ok so here's the deal... this is set up to define the Layers by either the URL of the Layer or the Item ID. Only use one of the options below. 

## Define Layer By URL

In [ ]:
LayerURLToReplace = input("What is the URL of the Layer you want to replace?")

## Define Layer By Item ID

In [ ]:
LayerIDToReplace = input("What is the Item ID of the Layer you want to replace?")
layerID_to_replace_item = gis.content.get(LayerIDToReplace)
print(layerID_to_replace_item)

# Step 4: Define the Replacement layer By Item ID
Ok so here's the deal... this is set up to define the Layers by the Item ID only.

In [ ]:
ReplacementLayerItemID = input("What is the Replacement Layer's ItemID? ")
replacement_layer_item = gis.content.get(ReplacementLayerItemID)
print(replacement_layer_item)

# Step 5: Define Functions

In [ ]:
def DefineMapDictionary(OperationalLayerJSON):
    MapDictionary = {}
    for index, layer in enumerate(OperationalLayerJSON):
        #print(index)
        #print (layer)
        LayerID = str(layer.get("id"))
        #print(LayerID)
        LayerName = str(layer.get("title")).replace(",", "")
        #print(LayerName)
        LayerItemID = str(layer.get('itemId'))
        #print(LayerItemID)
        LayerURL = str(layer.get('url'))
        #print(LayerURL)
        MapDictionary[LayerID]=[index]
        MapDictionary[LayerItemID]=[index]
        MapDictionary[LayerURL]=[index]
    return MapDictionary

def ReadDictionary(DictionaryToBeRead, SearchParameter):
    #DictionaryValueIndex = DictionaryToBeRead.get(SearchParameter)
    result = [val for key, val in DictionaryToBeRead.items() if SearchParameter in key]
    #print(result)
    if result != []:
        DictionaryValueIndex = result[0]
        return SearchParameter, DictionaryValueIndex
    else:
        return SearchParameter, None
    
print("Function Section Complete!")

# Step 6: Audit Web Maps and Replace Layers

In [ ]:
if LayerURLToReplace != "" and LayerIDToReplace != "":
    LayertoReplace = ""
    print("Define Layer to replace by only using one method.")
elif LayerURLToReplace != "":
    LayertoReplace = LayerURLToReplace
    LayertoReplaceMethod = "URL"
    print(LayertoReplace)
elif LayerIDToReplace != "":
    LayertoReplace = LayerIDToReplace
    LayertoReplaceMethod = "ItemID"
    print(LayertoReplace)
    
if ReplacementLayerItemID == "":
    ReplacmentLayer = ""
    print("Define Replacement Layer by Item ID.")
elif ReplacementLayerItemID != "":
    ReplacmentLayer = ReplacementLayerItemID
    ReplaceMethod = "ItemID"
    print(ReplacmentLayer)
    
if LayertoReplace != "" and ReplacmentLayer != "":
    print("Layers defined...")
    print("Start Searching...")
    Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000) # All Web Maps
    #Audit = gis.content.search(query="id:fcdditemidgoeshereff1", item_type="Web Map", max_items = 10000) # Item Id of a Web Map
    for Item in Audit:
        UpdateStatus = ""
        ItemTitle = str(Item["title"].replace(',', ""))
        ItemID = str(Item["id"])
        print(ItemID)
        ItemType = str(Item["type"])
        ItemOwner = str(Item["owner"])
        ItemAccess = str(Item["access"])
        ItemContentStatus = str(Item.get("contentStatus"))
        ItemData = Item.get_data(try_json=True)
        #print(ItemData)
        for ItemInfo in ItemData:
            if ItemInfo == "operationalLayers":
                layers = ItemData["operationalLayers"]
                OriginalMapDictionary = DefineMapDictionary(layers)
                #print(OriginalMapDictionary)
                for index, layer in enumerate(layers):
                    #print (layer)
                    s += ItemTitle.replace(",", "") + ","
                    s += ItemID + ","
                    s += ItemType + ","
                    s += ItemOwner + ","
                    s += ItemAccess + ","
                    s += "Operational Layer" + ","
                    s += str(index) + ","
                    LayerID = str(layer.get("id"))
                    s += LayerID + ","
                    #print(LayerID)
                    s += str(layer.get("title")).replace(",", "") + ","
                    s += str(layer.get("layerType")) + ","
                    s += str(layer.get("visibility")) + ","
                    LayerURL = str(layer.get('url'))
                    s += LayerURL + ","
                    #print(LayerURL)
                    LayerItemID = str(layer.get('itemId'))
                    #print(LayerItemID)
                    # Search Replacement Layer by Item ID
                    if LayertoReplaceMethod == "ItemID":
                        if LayerItemID != "None":
                            s += LayerItemID + ","
                            #LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                            if LayerItemID == LayertoReplace:
                                print("ready to replace layer")
                                webmap = WebMap(Item)
                                s += ReplacmentLayer + ","
                                layer_to_replace = webmap.get_layer(layer_id=LayerID)
                                #print(layer_to_replace)
                                replacement_layer = gis.content.get(ReplacmentLayer)
                                #print(replacement_layer)
                                webmap.remove_layer(layer_to_replace)
                                print("removedLayer")
                                webmap.add_layer(replacement_layer)
                                print("AddedLayer")
                                webmap.update()
                                print("updatedWebmap")
                                UpdateStatus = "Updated"
                                s += UpdateStatus + ","
                                print("UpdateStatus: " + UpdateStatus)
                            else:
                                s += ","
                                s += "NotUpdated" + ","
                                print("UpdateStatus: " + "NotUpdated")
                        else:
                            s += ","
                    # Search Replacement Layer by Item Url
                    elif LayertoReplaceMethod == "URL":
                        if LayerURL != "None":
                            s += LayerURL + ","
                            #LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                            if LayertoReplace in LayerURL:
                                print("ready to replace layer")
                                webmap = WebMap(Item)
                                s += ReplacmentLayer + ","
                                layer_to_replace = webmap.get_layer(layer_id=LayerID)
                                #print(layer_to_replace)
                                replacement_layer = gis.content.get(ReplacmentLayer)
                                #print(replacement_layer)
                                webmap.remove_layer(layer_to_replace)
                                print("removedLayer")
                                webmap.add_layer(replacement_layer)
                                print("AddedLayer")
                                webmap.update()
                                print("updatedWebmap")
                                UpdateStatus = "Updated"
                                s += UpdateStatus + ","
                                print("UpdateStatus: " + UpdateStatus)
                            else:
                                s += ","
                                s += "NotUpdated" + ","
                                print("UpdateStatus: " + "NotUpdated")
                        else:
                            s += ","
                    s += "\n"
        #outside webmap details loop
        if UpdateStatus == "Updated":
            print("Gathering info to reorder the map...")
            item = gis.content.get(ItemID)
            item_data = item.get_data()
            #print(item_data)
            AdjustmentMapDictionary = {}
            for ItemInfo in item_data:
                if ItemInfo == "operationalLayers":
                    layers = item_data["operationalLayers"]
                    UpdatedMapDictionary = DefineMapDictionary(layers)
                    #print(UpdatedMapDictionary)
                    for index, layer in enumerate(layers):
                        ## get the name of the layer
                        #print(index)
                        #print (layer)
                        LayerID = str(layer.get("id"))
                        #print(LayerID)
                        LayerName = str(layer.get("title")).replace(",", "")
                        #print(LayerName)
                        LayerItemID = str(layer.get('itemId'))
                        #print(LayerItemID)
                        LayerURL = str(layer.get('url'))
                        #print(LayerURL)

                        OriginalSearchLayer, OriginalSearchIndex = ReadDictionary(OriginalMapDictionary,LayerID)
                        #print(str(OriginalSearchLayer) + " " + str(OriginalSearchIndex))
                        UpdatedSearchLayer, UpdatedSearchIndex = ReadDictionary(UpdatedMapDictionary,LayerID)
                        if OriginalSearchIndex != None:
                            #print("original search index not none")
                            AdjustmentMapDictionary[OriginalSearchIndex[0]] = layers[UpdatedSearchIndex[0]]
                        else:
                            #Search the dictionaries by item id
                            #print("search by item id")
                            #print(OriginalMapDictionary)
                            OriginalSearchLayer, OriginalSearchIndex = ReadDictionary(OriginalMapDictionary,LayertoReplace)
                            #print(str(OriginalSearchLayer) + " " + str(OriginalSearchIndex))
                            AdjustmentMapDictionary[OriginalSearchIndex[0]] = layers[UpdatedSearchIndex[0]]

            #print(AdjustmentMapDictionary)
            SortedAdjustmentMapDictionary = dict(sorted(AdjustmentMapDictionary.items()))
            #print(SortedAdjustmentMapDictionary)
            for key, value in SortedAdjustmentMapDictionary.items():
                #print(key)
                #print(value)
                item_data['operationalLayers'][key] = value
                #print(OperationalLayer)
            item_properties = {"text":item_data}
            #print(item_properties)
            item.update(item_properties=item_properties)
            print("Item updated")  
        else:
            print("Item Skipped... Does not need to be updated.")

print("Audit Section Complete!") 

# Step 7: (optional) Write the Audit Results to csv

In [ ]:
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Write data To CSV File")
    output.write("MapTitle,MapId,ItemType,ItemOwner,ItemAccess,MapLocation,LayerIndex,LayerID,LayerTitle,LayerType,LayerVisisbility,LayerURL,ItemID,ReplacementLayerItemID,UpdateStatus\n")
    output.writelines(s)
    print("Write data To CSV File Complete!")